<a href="https://colab.research.google.com/github/Rajib2805/Covid__visualization/blob/main/ICVL_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**THE OPTIMIZATION MODEL IS FRAMED WITH THE OBJECTIVE OF HAVING A MONTHLY VIEW AND DERIVE ESTIMATES FOR THE MONTH AHEAD**

**Even though optimization problems are all about contraints in the decision making process, it must be remembered that constraints must be strived to be changed for the better with better operation and managerial decisions. It must be assumed that no constrains are God created and everything is changeable.**

In [3]:
!pip install pulp
import pulp as p

In [4]:
#Defining the problem
Lp_prob = p.LpProblem('Problem', p.LpMaximize)  

# Create problem Variables  
CCs = p.LpVariable('CCs', lowBound = 0) 
TCs = p.LpVariable('TCs', lowBound = 0) 
ROM_mined_perday = p.LpVariable('ROM_mined_perday', lowBound = 0) 
ROM_feed_perday = p.LpVariable('ROM_feed_perday', lowBound = 0) 
CCp = p.LpVariable('CCp', lowBound = 0) 
TCp = p.LpVariable('TCp', lowBound = 0) 
CCro = p.LpVariable('CCro', lowBound = 0) 
TCro = p.LpVariable('TCro', lowBound = 0) 
CCra = p.LpVariable('CCra', lowBound = 0) 
TCra = p.LpVariable('TCra', lowBound = 0) 
CClera = p.LpVariable('CClera', lowBound = 0) 
TClera = p.LpVariable('TClera', lowBound = 0) 
Stock_ROM = p.LpVariable('Stock_ROM', lowBound = 0) 
Stock_CHPP_CC = p.LpVariable('Stock_CHPP_CC', lowBound = 0) 
Stock_CHPP_TC = p.LpVariable('Stock_CHPP_TC', lowBound = 0) 
Stock_RS_CC = p.LpVariable('Stock_RS_CC', lowBound = 0) 
Stock_RS_TC = p.LpVariable('Stock_RS_TC', lowBound = 0) 
Stock_Port_CC = p.LpVariable('Stock_Port_CC', lowBound = 0) 
Stock_Port_TC = p.LpVariable('Stock_Port_TC', lowBound = 0) 
ccvessel = p.LpVariable('ccvessel', lowBound = 0, cat=p.LpInteger) 
tcvessel = p.LpVariable('tcvessel', lowBound = 0, cat=p.LpInteger) 


# Objective Function of profit earned per ton / (-) for loss incurred per ton
Lp_prob+= -25 * CCs + 25 * TCs   



# DIFFERENT CONSTRAINTS:
#Sales constraint as per EJC
Lp_prob += CCs >= 0 #taking the value >=0 shall give us the best decision without any pressure from EJC
Lp_prob += TCs >= 0 #taking the value >=0 shall give us the best decision without any pressure from EJC

#Minimum Cashflow generation requirement
Lp_prob += CCs * 92 + TCs * 63 >= 10000000

#Mining constraint
Lp_prob += Stock_ROM + ROM_mined_perday*30 >= ROM_feed_perday*30

#CHPP constraint of working days
Lp_prob += CCp <= 0.38 * ROM_feed_perday * 26
Lp_prob += TCp <= 0.07 * ROM_feed_perday * 26

#Road haulage constraint
Lp_prob += CCro + TCro >= 110000
Lp_prob += CCro + TCro <= 150000

#Rail transport constraint
Lp_prob += CCra + TCra + CClera + TClera >= 120000
Lp_prob += CCra + TCra + CClera + TClera <= 170000

#Vessel capacity constraint
Lp_prob += ccvessel >= CCs*1/38500
Lp_prob += tcvessel >= TCs*1/38500
Lp_prob += ccvessel <= CCs*1/38500
Lp_prob += tcvessel <= TCs*1/38500

#Stock flow constraint to avoid stock-out
Lp_prob += Stock_ROM + ROM_mined_perday*28 >= ROM_feed_perday*28
Lp_prob += Stock_CHPP_CC + CCp >= CCro
Lp_prob += Stock_CHPP_TC + TCp >= TCro
Lp_prob += Stock_RS_CC + CCro >= CCra + CClera 
Lp_prob += Stock_RS_TC + TCro >= TCra + TClera
Lp_prob += Stock_Port_CC + CCra + CClera >= CCs
Lp_prob += Stock_Port_TC + TCra + TClera >= TCs

#Minimum stock to be maintained at different stockyards as buffer stock
Lp_prob += Stock_ROM + ROM_mined_perday*28 - ROM_feed_perday*28 >= 50000
Lp_prob += (Stock_CHPP_CC + CCp - CCro) +(Stock_CHPP_TC + TCp - TCro) >= 40000
Lp_prob += (Stock_RS_CC + CCro - CCra - CClera) + (Stock_RS_TC + TCro - TCra- TClera) >= 40000
Lp_prob += (Stock_Port_CC + CCra + CClera - CCs) + (Stock_Port_TC + TCra + TClera - TCs) >= 40000

#Storage space constraint
Lp_prob += Stock_ROM + ROM_mined_perday *28 - ROM_feed_perday *28 <= 100000 # space at Mine
Lp_prob += Stock_CHPP_CC + Stock_CHPP_TC + CCp + TCp - CCro - TCro <= 90000 # space at CHPP  
Lp_prob += Stock_RS_CC + Stock_RS_TC + CCro + TCro - CCra - TCra - CClera - TClera<= 100000 # space at RS
Lp_prob += Stock_Port_CC + Stock_Port_TC + CCra + TCra + CClera + TClera - CCs - TCs <=150000 # space at Port

# Opening/Present Stock levels
Lp_prob += Stock_ROM <= 30000
Lp_prob += ROM_mined_perday <= 15000 #from regression
Lp_prob += Stock_CHPP_CC <= 25000
Lp_prob += Stock_CHPP_TC <= 500000
Lp_prob += Stock_RS_CC <= 40000
Lp_prob += Stock_RS_TC <= 10000
Lp_prob += Stock_Port_CC <= 60000
Lp_prob += Stock_Port_TC <= 25000

# Display the problem 
print(Lp_prob) 

#printing the status of the optimization problem
status = Lp_prob.solve()   # Solver CBC by default
print('The status of the problem is: ', p.LpStatus[status]) 

# Printing the final solution 
print('Profit value is: ', p.value(Lp_prob.objective))
print('ROM_feed is: ', p.value(ROM_feed_perday*26))
print('Production in CHPP CC: ', p.value(CCp), ', TC: ', p.value(TCp))
print('Road despatch CC: ', p.value(CCro), ', TC: ', p.value(TCro))
print('Rail despatch CC: ', p.value(CCra), ', TC: ', p.value(TCra))
print('Lease Rail despatch CC: ', p.value(CClera), ', TC: ', p.value(TClera))
print('Shipping/Sale CC: ', p.value(CCs), ', TC: ', p.value(TCs))
print('Number of vessels CC: ', p.value(ccvessel), ', TC: ', p.value(tcvessel))

print('Stock at Mine  ROM: ', p.value(Stock_ROM + ROM_mined_perday*26 - ROM_feed_perday*26))
print('Stock at CHPP  CC: ', p.value(Stock_CHPP_CC + CCp-CCro), 'TC: ', p.value(Stock_CHPP_TC + TCp-TCro))
print('Stock at RS  CC: ', p.value(Stock_RS_CC + CCro-CCra-CClera), 'TC: ', p.value(Stock_RS_TC + TCro-TCra-TClera))
print('Stock at Port  CC: ', p.value(Stock_Port_CC + CCra + CClera - CCs), 'TC: ', p.value(Stock_Port_TC + TCra +TClera - TCs))

Problem:
MAXIMIZE
-25*CCs + 25*TCs + 0
SUBJECT TO
_C1: CCs >= 0

_C2: TCs >= 0

_C3: 92 CCs + 63 TCs >= 10000000

_C4: - 30 ROM_feed_perday + 30 ROM_mined_perday + Stock_ROM >= 0

_C5: CCp - 9.88 ROM_feed_perday <= 0

_C6: - 1.82 ROM_feed_perday + TCp <= 0

_C7: CCro + TCro >= 110000

_C8: CCro + TCro <= 150000

_C9: CClera + CCra + TClera + TCra >= 120000

_C10: CClera + CCra + TClera + TCra <= 170000

_C11: - 2.5974025974e-05 CCs + ccvessel >= 0

_C12: - 2.5974025974e-05 TCs + tcvessel >= 0

_C13: - 2.5974025974e-05 CCs + ccvessel <= 0

_C14: - 2.5974025974e-05 TCs + tcvessel <= 0

_C15: - 28 ROM_feed_perday + 28 ROM_mined_perday + Stock_ROM >= 0

_C16: CCp - CCro + Stock_CHPP_CC >= 0

_C17: Stock_CHPP_TC + TCp - TCro >= 0

_C18: - CClera - CCra + CCro + Stock_RS_CC >= 0

_C19: Stock_RS_TC - TClera - TCra + TCro >= 0

_C20: CClera + CCra - CCs + Stock_Port_CC >= 0

_C21: Stock_Port_TC + TClera + TCra - TCs >= 0

_C22: - 28 ROM_feed_perday + 28 ROM_mined_perday + Stock_ROM >= 50000

_